In [9]:
import math
import collections

import pandas as pd

from keras import regularizers
from keras.models import Model, Sequential
from keras.layers import *

import params
from utils.sequence_data import generator_for_binary_classifier, generator_to_samples_and_targets
from utils.metrics import print_report_for_binary_classfier
from utils.preprocessing import probs_to_binary_classes
from utils.hyperparams import hyperparams_search
from utils.plot import plot_train_validation_metric

In [10]:
dataset = pd.read_csv("../datasets/data_for_models/dataset_1996-01-01_2019-08-22.csv",
                           header=0, parse_dates=[0], index_col=0)
input_shape = (None, dataset.shape[-1])
#params for generator
label_index = len(dataset.columns) -1

train_max_idx = math.ceil(len(dataset)*params.TRAIN_RATIO)
val_max_idx = math.ceil(len(dataset)*(params.TRAIN_RATIO+params.VAL_RATIO))

# 1 step = 1 batche of samples 
train_steps = (train_max_idx+1 -params.LOOKBACK) // params.BATCH_SIZE
val_steps =  (val_max_idx - train_max_idx - params.LOOKBACK) // params.BATCH_SIZE
test_steps = (len(dataset) - val_max_idx - params.LOOKBACK) // params.BATCH_SIZE



In [11]:
#init generator_for_binary_classifiers
train_gen = generator_for_binary_classifier(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)



val_gen = generator_for_binary_classifier(dataset.to_numpy(),
                    label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=train_max_idx+1,
                      max_index=val_max_idx,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)

test_gen = generator_for_binary_classifier(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=params.LOOKBACK,
                      delay=params.DELAY,
                      min_index=val_max_idx+1,
                      max_index=None,
                      shuffle=False,
                      step=params.STEP, 
                      batch_size=params.BATCH_SIZE,
                      interval_label=True)

In [13]:
def lstm_model(hidden_unit, n_layer, l2_weight, input_shape):
    model = Sequential()
    is_return_sequences = True if n_layer>=2 else False
    
    # first layer
    model.add(LSTM(hidden_unit, 
                   input_shape=input_shape,
                   kernel_regularizer=regularizers.l2(l2_weight),
                   recurrent_regularizer=regularizers.l2(l2_weight),
                   return_sequences=is_return_sequences))
    
    # intermediate layers
    if n_layer >= 3:
        for _ in range(n_layer-2):
            model.add(LSTM(hidden_unit,
                           kernel_regularizer=regularizers.l2(l2_weight),
                           recurrent_regularizer=regularizers.l2(l2_weight),
                           return_sequences=is_return_sequences))

    
    # last layer
    if n_layer >= 2:
        model.add(LSTM(hidden_unit,
                       recurrent_regularizer=regularizers.l2(l2_weight),
                       kernel_regularizer=regularizers.l2(l2_weight)))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
best_params = hyperparams_search(model_func=lstm_model, 
                                 input_shape=input_shape,
                                 train_steps=train_steps, 
                                 val_steps=val_steps, 
                                 train_gen=train_gen, 
                                 val_gen=val_gen, 
                                 num_epoch=1000,
                                 hidden_units=[32,64], 
                                 n_layers=[1,2], 
                                 l2_weights=[0,0.0005,0.0015],
                                 is_verbose=1)
print(best_params)

Epoch 1/1000
63/63 [==============================] - 4s 60ms/step - loss: 0.5766 - acc: 0.7560 - val_loss: 0.2755 - val_acc: 0.9868
Epoch 2/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.5668 - acc: 0.7517 - val_loss: 0.2548 - val_acc: 0.9868
Epoch 3/1000
63/63 [==============================] - 1s 20ms/step - loss: 0.5679 - acc: 0.7470 - val_loss: 0.2512 - val_acc: 0.9868
Epoch 4/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.5597 - acc: 0.7498 - val_loss: 0.2527 - val_acc: 0.9868
Epoch 5/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.5549 - acc: 0.7480 - val_loss: 0.2610 - val_acc: 0.9868
Epoch 6/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.5441 - acc: 0.7488 - val_loss: 0.2720 - val_acc: 0.9868
Epoch 7/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.5244 - acc: 0.7532 - val_loss: 0.2720 - val_acc: 0.9868
Epoch 8/1000
63/63 [==============================] - 2s 26ms/step - 

63/63 [==============================] - 1s 14ms/step - loss: 0.4277 - acc: 0.8147 - val_loss: 0.1842 - val_acc: 0.9423
Epoch 63/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4245 - acc: 0.8202 - val_loss: 0.1583 - val_acc: 0.9700
Epoch 64/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4307 - acc: 0.8058 - val_loss: 0.1435 - val_acc: 0.9531
Epoch 65/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4295 - acc: 0.8026 - val_loss: 0.1659 - val_acc: 0.9832
Epoch 66/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4187 - acc: 0.8115 - val_loss: 0.1512 - val_acc: 0.9555
Epoch 67/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4014 - acc: 0.8247 - val_loss: 0.1503 - val_acc: 0.9579
Epoch 68/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.4043 - acc: 0.8212 - val_loss: 0.1450 - val_acc: 0.9543
Epoch 69/1000
63/63 [==============================] - 1s 15ms/step - loss: 

63/63 [==============================] - 1s 18ms/step - loss: 0.2759 - acc: 0.8854 - val_loss: 0.3650 - val_acc: 0.8678
Epoch 124/1000
63/63 [==============================] - 1s 21ms/step - loss: 0.2680 - acc: 0.8859 - val_loss: 0.2779 - val_acc: 0.8918
Epoch 125/1000
63/63 [==============================] - 1s 19ms/step - loss: 0.2562 - acc: 0.8889 - val_loss: 0.2994 - val_acc: 0.8942
Epoch 126/1000
63/63 [==============================] - 1s 17ms/step - loss: 0.2806 - acc: 0.8909 - val_loss: 0.2858 - val_acc: 0.8906
Epoch 127/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.2801 - acc: 0.8802 - val_loss: 0.2121 - val_acc: 0.9171
Epoch 128/1000
63/63 [==============================] - 1s 20ms/step - loss: 0.2689 - acc: 0.8822 - val_loss: 0.3696 - val_acc: 0.8450
Epoch 129/1000
63/63 [==============================] - 1s 22ms/step - loss: 0.2903 - acc: 0.8785 - val_loss: 0.2368 - val_acc: 0.9255
Epoch 130/1000
63/63 [==============================] - 2s 27ms/step -

Epoch 184/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.1572 - acc: 0.9375 - val_loss: 0.8961 - val_acc: 0.7969
Epoch 185/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.1463 - acc: 0.9430 - val_loss: 0.7461 - val_acc: 0.7837
Epoch 186/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.1464 - acc: 0.9430 - val_loss: 0.7993 - val_acc: 0.8005
Epoch 187/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.1471 - acc: 0.9425 - val_loss: 0.7726 - val_acc: 0.8065
Epoch 188/1000
63/63 [==============================] - 1s 21ms/step - loss: 0.1422 - acc: 0.9482 - val_loss: 0.7329 - val_acc: 0.8113
Epoch 189/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.1474 - acc: 0.9437 - val_loss: 0.7987 - val_acc: 0.7873
Epoch 190/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.1477 - acc: 0.9407 - val_loss: 0.7159 - val_acc: 0.8233
Epoch 191/1000
63/63 [==============================] -

Epoch 245/1000
63/63 [==============================] - 1s 13ms/step - loss: 0.1018 - acc: 0.9621 - val_loss: 0.7506 - val_acc: 0.7885
Epoch 246/1000
63/63 [==============================] - 1s 13ms/step - loss: 0.0918 - acc: 0.9673 - val_loss: 0.7058 - val_acc: 0.8041
Epoch 247/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0754 - acc: 0.9720 - val_loss: 1.0026 - val_acc: 0.7764
Epoch 248/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.2062 - acc: 0.9239 - val_loss: 0.5616 - val_acc: 0.7608
Epoch 249/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.1412 - acc: 0.9474 - val_loss: 0.9850 - val_acc: 0.7572
Epoch 250/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0938 - acc: 0.9660 - val_loss: 0.7108 - val_acc: 0.8065
Epoch 251/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0877 - acc: 0.9702 - val_loss: 0.9140 - val_acc: 0.7885
Epoch 252/1000
63/63 [==============================] -

Epoch 306/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.0352 - acc: 0.9903 - val_loss: 1.6458 - val_acc: 0.6887
Epoch 307/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.0347 - acc: 0.9898 - val_loss: 1.7095 - val_acc: 0.6538
Epoch 308/1000
63/63 [==============================] - 1s 18ms/step - loss: 0.0291 - acc: 0.9931 - val_loss: 1.6400 - val_acc: 0.6827
Epoch 309/1000
63/63 [==============================] - 1s 19ms/step - loss: 0.0393 - acc: 0.9886 - val_loss: 1.7836 - val_acc: 0.6418
Epoch 310/1000
63/63 [==============================] - 1s 17ms/step - loss: 0.1277 - acc: 0.9514 - val_loss: 0.9048 - val_acc: 0.7812
Epoch 311/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.1959 - acc: 0.9340 - val_loss: 0.5613 - val_acc: 0.8377
Epoch 312/1000
63/63 [==============================] - 1s 17ms/step - loss: 0.2097 - acc: 0.9246 - val_loss: 0.5534 - val_acc: 0.8017
Epoch 313/1000
63/63 [==============================] -

Epoch 367/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0369 - acc: 0.9874 - val_loss: 1.7656 - val_acc: 0.6346
Epoch 368/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0674 - acc: 0.9722 - val_loss: 1.3528 - val_acc: 0.7332
Epoch 369/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0647 - acc: 0.9759 - val_loss: 1.4501 - val_acc: 0.7212
Epoch 370/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0304 - acc: 0.9916 - val_loss: 1.6448 - val_acc: 0.6514
Epoch 371/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0207 - acc: 0.9958 - val_loss: 1.6571 - val_acc: 0.6755
Epoch 372/1000
63/63 [==============================] - 1s 15ms/step - loss: 0.0133 - acc: 0.9978 - val_loss: 1.6404 - val_acc: 0.6887
Epoch 373/1000
63/63 [==============================] - 1s 14ms/step - loss: 0.0134 - acc: 0.9983 - val_loss: 1.7057 - val_acc: 0.6815
Epoch 374/1000
63/63 [==============================] -

63/63 [==============================] - 1s 18ms/step - loss: 0.0147 - acc: 0.9968 - val_loss: 2.1279 - val_acc: 0.6478
Epoch 428/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.0332 - acc: 0.9883 - val_loss: 1.8685 - val_acc: 0.6791
Epoch 429/1000
63/63 [==============================] - 1s 18ms/step - loss: 0.0414 - acc: 0.9866 - val_loss: 1.6433 - val_acc: 0.6815
Epoch 430/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.1914 - acc: 0.9358 - val_loss: 1.1560 - val_acc: 0.7392
Epoch 431/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.0821 - acc: 0.9690 - val_loss: 1.4156 - val_acc: 0.7476
Epoch 432/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.0705 - acc: 0.9735 - val_loss: 1.5769 - val_acc: 0.7091
Epoch 433/1000
63/63 [==============================] - 1s 16ms/step - loss: 0.0372 - acc: 0.9886 - val_loss: 1.3038 - val_acc: 0.7644
Epoch 434/1000
63/63 [==============================] - 1s 17ms/step -

In [ ]:
#hyperparameters
num_epoch = best_params['epochs'] 
n_layer= best_params['n_layer'] 
hidden_unit=best_params['hidden_unti']  
l2_weight=best_params['l2_weight']

model = lstm_model(hidden_unit, n_layer, l2_weight, input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# #fit the model
history = model.fit_generator(train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=epochs, verbose=1)



In [ ]:
plot_train_validation_metric(history, 'loss', 'val_loss')

## We have 2984 positive and 984 negtive cases. 

In [ ]:
#Make predictions for train set
    
X, Y = generator_to_samples_and_targets(train_gen, train_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

In [ ]:
#Make predictions from dev set
X, Y = generator_to_samples_and_targets(val_gen, val_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

In [ ]:
#Make predictions for test set
X, Y = generator_to_samples_and_targets(test_gen, test_steps)    
Y_pred = model.predict(X)
print(collections.Counter(Y))
print_report_for_binary_classfier(Y, probs_to_binary_classes(Y_pred))

# Save the best model

In [ ]:
from keras.models import load_model

model.save(params.BEST_MODEL_PATH)  # creates a HDF5 file 'my_model.h5'